In [2]:
import pandas as pd
import sys
sys.path.insert(1, '../../scripts/')
from s3_support import *

# 2023 updates

In [52]:
q = '''select
            org,
            sum(amount) as sum,
            count(distinct(id)) as count
        from transactions
        where
            status='A' and
            (date='2022-11-29' or (date='2022-11-30' and hour<3))
        group by org'''

orgs_2022 = redshift_query_read(q, schema='production')

In [53]:
q = '''select
            org,
            sum(amount) as sum,
            count(distinct(id)) as count
        from transactions
        where
            status='A' and
            (date='2023-11-28' or (date='2023-11-29' and hour<3))
        group by org'''

orgs_2023 = redshift_query_read(q, schema='production')

In [54]:
_orgs_2022 = orgs_2022[orgs_2022['sum']>500]
_orgs_2023 = orgs_2023[orgs_2023['sum']>500]

print("{:,} orgs 2022 > $500; ${:,.2f}".format(len(_orgs_2022), _orgs_2022['sum'].sum()))
print("{:,} orgs 2023 > $500; ${:,.2f}".format(len(_orgs_2023), _orgs_2023['sum'].sum()))

print("{:,} orgs in 2023 also in 2022".format(len(_orgs_2023[_orgs_2023['org'].isin(_orgs_2022['org'].tolist())])))

overlap_orgs = _orgs_2023[_orgs_2023['org'].isin(_orgs_2022['org'].tolist())]['org'].tolist()

print("Overlap orgs 2022: ${:,.2f}".format(_orgs_2022[_orgs_2022['org'].isin(overlap_orgs)]['sum'].sum()))
print("Overlap orgs 2023: ${:,.2f}".format(_orgs_2023[_orgs_2023['org'].isin(overlap_orgs)]['sum'].sum()))

print("Overlap orgs 2022 median: ${:,.2f}".format(_orgs_2022[_orgs_2022['org'].isin(overlap_orgs)]['sum'].median()))
print("Overlap orgs 2023 median: ${:,.2f}".format(_orgs_2023[_orgs_2023['org'].isin(overlap_orgs)]['sum'].median()))

print("Overlap orgs 2022 transactions: {:,}".format(_orgs_2022[_orgs_2022['org'].isin(overlap_orgs)]['count'].sum()))
print("Overlap orgs 2023 transactions: {:,}".format(_orgs_2023[_orgs_2023['org'].isin(overlap_orgs)]['count'].sum()))

1,006 orgs 2022 > $500; $6,299,339.70
992 orgs 2023 > $500; $6,689,775.11
621 orgs in 2023 also in 2022
Overlap orgs 2022: $5,265,082.45
Overlap orgs 2023: $5,330,925.72
Overlap orgs 2022 median: $3,241.75
Overlap orgs 2023 median: $3,170.50
Overlap orgs 2022 transactions: 33,839
Overlap orgs 2023 transactions: 27,714


In [51]:
_orgs_2022 = orgs_2022[orgs_2022['sum']>25000]
_orgs_2023 = orgs_2023[orgs_2023['sum']>25000]

print("{:,} orgs 2022 > $25k; ${:,.2f}".format(len(_orgs_2022), _orgs_2022['sum'].sum()))
print("{:,} orgs 2023 > $25k; ${:,.2f}".format(len(_orgs_2023), _orgs_2023['sum'].sum()))

print("{:,} orgs in 2023 also in 2022".format(len(_orgs_2023[_orgs_2023['org'].isin(_orgs_2022['org'].tolist())])))

overlap_orgs = _orgs_2023[_orgs_2023['org'].isin(_orgs_2022['org'].tolist())]['org'].tolist()

print("Overlap orgs 2022: ${:,.2f}".format(_orgs_2022[_orgs_2022['org'].isin(overlap_orgs)]['sum'].sum()))
print("Overlap orgs 2023: ${:,.2f}".format(_orgs_2023[_orgs_2023['org'].isin(overlap_orgs)]['sum'].sum()))

print("Overlap orgs 2022 median: ${:,.2f}".format(_orgs_2022[_orgs_2022['org'].isin(overlap_orgs)]['sum'].median()))
print("Overlap orgs 2023 median: ${:,.2f}".format(_orgs_2023[_orgs_2023['org'].isin(overlap_orgs)]['sum'].median()))

50 orgs 2022 > $25k; $2,674,293.17
46 orgs 2023 > $25k; $2,879,813.08
26 orgs in 2023 also in 2022
Overlap orgs 2022: $1,792,614.97
Overlap orgs 2023: $1,947,965.26
Overlap orgs 2022 median: $42,960.79
Overlap orgs 2023 median: $55,348.22


In [34]:
print("{:,} orgs 2022".format(len(orgs_2022)))
print("{:,} orgs 2023".format(len(orgs_2023)))

print("{:,} orgs in 2023 also in 2022".format(len(orgs_2023[orgs_2023['org'].isin(orgs_2022['org'].tolist())])))
print("{:,} orgs in 2022 also in 2023".format(len(orgs_2022[orgs_2022['org'].isin(orgs_2023['org'].tolist())])))

1,915 orgs 2022
1,792 orgs 2023
1,297 orgs in 2023 also in 2022
1,297 orgs in 2022 also in 2023


In [17]:
q = '''select
            count(distinct(org)) as orgs,
            count(distinct(id)) as count,
            count(distinct(case when recurring_origin=1 then id else null end)) as recurring_count,
            sum(amount) as volume,
            max(amount) as max_amount,
            avg(amount) as mean_amount
        from transactions
        where
            status='A' and
            (date='2023-11-28' or (date='2023-11-29' and hour<3))'''
print("2023 totals:")
redshift_query_read(q, schema='production')

2023 totals:


,orgs,count,recurring_count,volume,max_amount,mean_amount
0,1792,35742,571,6828754.13,30000.0,191.056856


In [3]:
q = '''select
            date,
            hour,
            count(distinct(org)) as orgs,
            count(distinct(id)) as count,
            sum(amount) as volume
        from transactions
        where
            status='A' and
            (recurring=0 or recurring_origin=1) and
            (date='2023-11-28' or (date='2023-11-29' and hour<5))
        group by date, hour'''
df = redshift_query_read(q, schema='production')
df.sort_values(['date', 'hour'], ascending=True)

,date,hour,orgs,count,volume
19,2023-11-28,0,77,96,14133.46
1,2023-11-28,1,52,67,13380.95
2,2023-11-28,2,29,32,8372.14
23,2023-11-28,3,21,25,3321.38
6,2023-11-28,4,31,33,5420.43
0,2023-11-28,5,57,76,16312.74
5,2023-11-28,6,144,287,54049.84
14,2023-11-28,7,239,604,121489.17
20,2023-11-28,8,422,1317,220416.67
9,2023-11-28,9,545,1916,361435.24


In [23]:
q = '''select
            median(amount)
        from transactions
        where
            status='A' and
            recurring=0 and
            (date='2023-11-28' or (date='2023-11-29' and hour<3))'''
redshift_query_read(q, schema='production')

,median
0,70.0


# <= 2022

In [24]:
q = '''select
            median(amount)
        from transactions
        where
            status='A' and
            recurring=0 and
            (date='2022-11-29' or (date='2022-11-30' and hour<3))'''
redshift_query_read(q, schema='production')

,median
0,52.5


In [27]:
q = '''select
            count(distinct(org)) as orgs,
            count(distinct(id)) as count,
            count(distinct(case when recurring_origin=1 then id else null end)) as recurring_count,
            sum(amount) as volume,
            max(amount) as max_amount,
            avg(amount) as mean_amount
        from transactions
        where
            status='A' and
            (date='2022-11-29' or (date='2022-11-30' and hour<3))'''
print("2022 totals:")
redshift_query_read(q, schema='production')

2022 totals:


,orgs,count,recurring_count,volume,max_amount,mean_amount
0,1915,42165,730,6453636.96,50000.0,153.056729


In [30]:
df['volume'].sum()

6453636.96

In [29]:
q = '''select
            date,
            hour,
            count(distinct(org)) as orgs,
            count(distinct(id)) as count,
            sum(amount) as volume
        from transactions
        where
            status='A' and
            (date='2022-11-29' or (date='2022-11-30' and hour<3))
        group by date, hour'''
df = redshift_query_read(q, schema='production')
df.sort_values(['date', 'hour'], ascending=True)

,date,hour,orgs,count,volume
22,2022-11-29,0,62,83,15529.17
3,2022-11-29,1,52,65,10355.44
11,2022-11-29,2,38,47,5648.84
1,2022-11-29,3,22,27,3112.59
18,2022-11-29,4,36,42,5492.07
5,2022-11-29,5,48,67,12461.87
23,2022-11-29,6,174,343,45818.58
0,2022-11-29,7,248,539,100843.40
17,2022-11-29,8,412,1198,228189.15
12,2022-11-29,9,520,1829,349325.24


In [11]:
q = '''select
            date,
            hour,
            count(distinct(org)) as orgs,
            count(distinct(id)) as count,
            sum(amount) as volume
        from transactions
        where
            status='A' and
            (date='2021-11-30' or (date='2021-12-01' and hour<3))
        group by date, hour'''
df = redshift_query_read(q, schema='production')
df.sort_values(['date', 'hour'], ascending=True)[['orgs', 'count', 'volume']]

,orgs,count,volume
21,73,107,18964.64
3,44,62,21899.21
23,35,54,6146.12
6,26,39,28917.43
15,26,36,3147.53
0,52,88,16042.49
12,294,1100,86514.23
10,590,3686,256143.53
24,766,4192,415859.59
22,659,2835,404326.13


# location

In [16]:
q = '''select
            state,
            count(distinct(org)) as orgs,
            count(distinct(id)) as count,
            sum(amount) as volume
        from transactions_clean
        where
            status='A' and
            (date='2022-11-29' or (date='2022-11-30' and hour<3))
        group by state'''
df = redshift_query_read(q, schema='production')
df.sort_values(['volume'], ascending=False)[:50]

,state,orgs,count,volume
182,CA,404,4195,711441.65
22,FL,409,2875,522613.97
51,PA,229,3503,509613.20
101,TX,292,2007,330676.59
88,None,521,2216,329747.85
183,NC,210,1796,320664.83
24,NY,304,2230,315560.77
217,MD,173,1621,238895.84
25,CO,180,1247,230919.55
92,OH,187,1295,223329.45
